In [9]:
import os
import sys
import csv
import gzip
import tqdm

import scipy.stats as stats

from collections import defaultdict

In [2]:
ls ../data/

drugReactionCounts.csv.gz
drugpairList.csv
drugpairReactionCounts.csv.gz
drugpairs.csv.gz
effect_sider.txt.gz
ingredientList.csv
omop_reference_effects.csv
project_aers_10q4.pred_drug_events_e5.txt.gz
reactionList.csv
stitch_to_rxnorm.csv


In [33]:
fh = '../data/project_aers_10q4.pred_drug_events_e5.txt.gz'
wcres = !gzcat $fh | wc -l
nlines = int(wcres[0])
reader = csv.reader(gzip.open(fh), delimiter='\t')
header = reader.next()
#print header

drugs = set()
events = set()

means = defaultdict(dict)

for row in tqdm.tqdm(reader, total=nlines):
    data = dict(zip(header, row))
    drugs.add(data['stitch_id'])
    events.add(data['umls_id'])
    means[data['umls_id']][data['stitch_id']]= float(data['drug_mean'])

100%|█████████▉| 1823954/1823955 [00:17<00:00, 106775.35it/s]


In [34]:
events = sorted(events)
drugs = sorted(drugs)

len(events), len(drugs)

(10437, 1332)

In [ ]:
correlations = dict()
for i, e1 in tqdm.tqdm(enumerate(events), total=len(events)):
    for j, e2 in enumerate(events):
        if j <= i:
            continue
        
        e1drugs = set(means[e1].keys())
        e2drugs = set(means[e2].keys())
        common_drugs = sorted(e1drugs & e2drugs)
        
        mus1 = [means[e1][d] for d in common_drugs]
        mus2 = [means[e2][d] for d in common_drugs]
        r, p = stats.pearsonr(mus1, mus2)
        
        correlations[(e1,e2)] = (r, p, len(common_drugs))

 10%|█         | 1062/10437 [24:31<3:36:28,  1.39s/it]